# Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, to_date, when, date_format, month


In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Covid") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/08/22 21:39:18 WARN Utils: Your hostname, Mohameds-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/08/22 21:39:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/22 21:39:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("./covid.csv", header=True, inferSchema=True)

# The Day with max covid cases
1. Group by date
2. With agg sum of Confirmed cases
3. Order by the sum
4. Get the first one

In [4]:
daily_cases = df.groupBy("Date").agg(sum("Total Confirmed cases").alias("Total_Cases"))

In [5]:
max_cases_day = daily_cases.orderBy(daily_cases["Total_Cases"].desc()).limit(1)
max_cases_day  = max_cases_day.withColumn("Formatted_Date", date_format(col("Date"), "dd-MMM-yyyy"))
result = max_cases_day.collect()[0]

formatted_date_str = result['Formatted_Date']
print(f"It was on {formatted_date_str} max covid cases recorded")


It was on 06-Aug-2020 max covid cases recorded


# The state has the second-largest number of covid cases.
- Group by State
- Find the agg sum of confirmed cases
- Order by 


In [6]:

state_cases = df.groupBy("Name of State / UT").agg(sum("Total Confirmed cases").alias("Total_Cases"))


In [7]:
second_largest_state = state_cases.orderBy(state_cases["Total_Cases"].desc()).limit(2).collect()[1]

print(f"State with second largest covid cases {second_largest_state['Name of State / UT']}")


State with second largest covid cases Tamil Nadu


# Which Union Territory has the least number of death.
1. Group by all the union territories with agg sum of deaths (No way to find UT)
2. Order by the the death count
3. get the first one

In [8]:
states = [
    "Andaman and Nicobar Islands", "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi", "Jammu and Kashmir", "Ladakh", "Lakshadweep", "Puducherry"
]

pattern = "(?i)" + "|".join(states)

states = df.filter(df["Name of State / UT"].rlike(pattern))


In [9]:
deaths = states.groupBy("Name of State / UT").agg(sum("Death").alias("Total_Deaths"))

In [10]:


state_with_low_deaths = deaths.orderBy("Total_Deaths").first()
state_with_low_deaths_non_zero = deaths.filter(col("Total_Deaths") > 0).orderBy("Total_Deaths").first()

print(f"Least deaths recorded in '{state_with_low_deaths_non_zero["Name of State / UT"]}'")

Least deaths recorded in 'Dadra and Nagar Haveli and Daman and Diu'


# The state has the lowest Death to Total Confirmed cases ratio.
1. Group by state
2. Calculate the agg sum death
3. Calculate the agg sum Total Cases
4. Find the ratio
5. Order by the ratio
6. Get the first one


In [11]:

state_ratios = df.groupBy("Name of State / UT").agg(
    sum("Death").alias("Total_Deaths"),
    sum("Total Confirmed cases").alias("Total_Cases")
).withColumn(
    "Death_to_Cases_Ratio",
    when(col("Total_Cases") != 0, col("Total_Deaths") / col("Total_Cases")).otherwise(None)
)


In [12]:
lowest_ratio_state = state_ratios.orderBy("Death_to_Cases_Ratio").first()

print(lowest_ratio_state["Name of State / UT"])

Mizoram


# Find which month the Most Newer recovered cases.
1. Group by months from dates
2. find the agg sum of new cases
3. Order the DF by agg sum
4. Get the first one

In [13]:

new_cases_by_month = df.groupBy(date_format('Date','MMMM').alias("Month")).agg(sum("New recovered").alias("Total New Recovered"))


In [14]:
most_recover_cases = new_cases_by_month.orderBy(new_cases_by_month["Total New Recovered"].desc()).first()["Month"]
print(f"The month with most new recoveries is '{most_recover_cases}'")

The month with most new recoveries is 'July'
